# Code for comparing models of the QPO in 1ES 1927+654 (e.g. computation of WD mass accretion rate)

In [1]:
# load packages
import numpy as np 
import matplotlib.pyplot as plt
import scipy.optimize as opt
import scipy.integrate as integ
import scipy.fft
import os
import astropy.stats as st
from astropy import units as u
from astropy import constants as const
from astropy.io import fits
from astropy.time import Time
from astropy.table import Table
from astropy.coordinates import SkyCoord
import pandas as pd
from matplotlib.ticker import ScalarFormatter, FormatStrFormatter, NullFormatter
from matplotlib import ticker
from matplotlib.colors import Normalize
import json
import glob
# from redshift get d_L
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70 * u.km / u.s / u.Mpc, Om0=0.3)
# emcee stuff
import multiprocessing as mp
# from multiprocessing import Pool
import emcee
import corner
import pylag
from joblib import Parallel, delayed
from scipy.stats import binned_statistic
import scipy.stats as stats

from timing import *

# set up plotting defaults
plt.rc('font', family='sans')
params = {
   'axes.labelsize': 45,
   'axes.linewidth': 3,
   'legend.fontsize': 30,
   'legend.frameon': True,
   'legend.fancybox': False,
   'legend.framealpha': 0.8,
   'legend.edgecolor': 'k',
   'lines.linewidth': 2,
   'font.size': 40,
   'font.weight': 'normal',
   'xtick.direction': 'in',
   'xtick.labelsize': 35,
   'xtick.color':'k',
   'xtick.major.bottom': True,
   'xtick.major.pad': 10,
   'xtick.major.size': 18,
   'xtick.major.width': 2,
   'xtick.minor.bottom': True,
   'xtick.minor.pad': 10,
   'xtick.minor.size': 9,
   'xtick.minor.top': True,
   'xtick.minor.visible': True,
   'xtick.minor.width': 2,
   'xtick.top': True,
   'ytick.direction': 'in',
   'ytick.labelsize': 35,
   'ytick.left': True,
   'ytick.right': True,
   'ytick.major.pad': 10,
   'ytick.major.size': 18,
   'ytick.major.width': 2,
   'ytick.minor.pad': 3.5,
   'ytick.minor.size': 9,
   'ytick.minor.visible': True,
   'ytick.minor.width': 2,
   'text.usetex': False,
   'figure.figsize': [10,10],
   'savefig.dpi': 500,
   }
plt.rcParams.update(params)

fig_path = '../figures/'
data_path = '../data/lightcurves/'
mcmc_path_bb = 'broadband_mcmcs/'
mcmc_path_qpo = 'qpo_mcmcs/'

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


## Mass Transfer Rates for WD Accretion

In [4]:
# Equation 8/9 -- assume fdot = 0 and solve for M2dot
M1 = 1e6 * const.M_sun 
M2 = 0.1 * const.M_sun
Mtot = M1 + M2
q = M2 / M1
f = 2.34 * u.mHz
M2dot = (M2 / (3 * (1 - q) + np.sqrt(1 + q))) * (96 * const.G**(5/3) * M1 * M2 * (2 * np.pi * f)**(8/3) / (5 * const.c**5 * Mtot**(1/3)))
M2dot_Msunperyear = (M2dot / const.M_sun).to(1/u.year).value
print('Required mass transfer rate to offset GR at f = 2.34 mHz: {:.1e} solar masses / year'.format(M2dot_Msunperyear))

Required mass transfer rate to offset GR at f = 2.34 mHz: 2.8e-04 solar masses / year
